# Using SPARQL and the Knowledge Graph for RAG
Now that the Knowledge Graph has been created we will use it to do local and global search.

![](../images/RAG_with_KnowledgeGraph.png)

In [0]:
## <add> install libraries and get config

In [0]:
%pip install databricks-vectorsearch databricks-langchain==0.8.1 -q

In [0]:
%pip install SPARQLWrapper openai -q

In [0]:
%pip install -U "langchain-openai>=0.2.0" "langchain-core>=0.2.0" "langchain>=0.2.0" "openai>=1.42.0" tiktoken -q

In [0]:
dbutils.library.restartPython()

In [0]:
%run ./00_config

In [0]:
import pandas as pd
import os
import urllib.parse
import ast
from io import StringIO
from SPARQLWrapper import SPARQLWrapper, CSV, SELECT, POST, POSTDIRECTLY
import os
import numpy as np
from typing import Dict, Any, List
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# from elasticsearch import Elasticsearch  # <delete>Commented out because Elasticsearch is not used
import json
from tqdm import tqdm

In [0]:
# Adjust pandas display settings
pd.set_option(
    "display.max_colwidth", None
)  # Set to None to display the full column width
pd.set_option("display.max_columns", None)

In [0]:
# <modify>Change to retrieve from `00_config`
# endpoint for GraphDB
# endpoint = "http://localhost:7200/repositories/msft-graphrag-300"

endpoint = f"{host}/repositories/{repository}"

In [0]:
es_username = 'elastic'
es_password = ''  # put your password here

In [0]:
def sparql_query(query: str, post=False) -> pd.DataFrame:
    sparql_conn.setQuery(query)
    sparql_conn.setReturnFormat(CSV)
    results = sparql_conn.query().convert()
    return pd.read_csv(StringIO(results.decode('utf-8')), sep=",")

In [0]:
sparql_conn = SPARQLWrapper(endpoint)

# <add> Set credentials for GraphDB 
sparql_conn.setCredentials(username, password)

## Setup the question
The question we will ask will be:
```
What is the relationship between Bob Cratchit and Belinda Cratchit?
```

In [0]:
question_text = "What is the relationship between Bob Cratchit and Belinda Cratchit?"

## Convert the question text
First step is to convert the question into an embedding vector. To do this we will use our local LM Studio instance and call the embedding OpenAI endpoint.

In [0]:
# <modify> Change to Databricks Mosaic AI Model Serving processing
# def get_embedding(text: str, client: Any, model: str="CompendiumLabs/bge-large-en-v1.5-gguf"):
#     """Convert the text into an embedding vector using the model provided

#     :param text: text to be converted to and embedding vector
#     :param client: OpenAI client
#     :param model: name of the model to use for encoding
#     """
#     text = text.replace("\n", " ")
#     return client.embeddings.create(input = [text], model=model).data[0].embedding

import os
from openai import OpenAI


def get_clinet():
    token = (
        dbutils.notebook.entry_point.getDbutils()
        .notebook()
        .getContext()
        .apiToken()
        .get()
    )

    # Databricks Workspace の URL をセット
    ws_url = "https://"
    ws_url += spark.conf.get("spark.databricks.workspaceUrl")

    return OpenAI(
        api_key=token,
        base_url=f"{ws_url}/serving-endpoints",
    )

def get_embedding(text, client, model="databricks-bge-large-en"):
    embeddings = client.embeddings.create(
        input=text,
        model=model,
    )

    return embeddings.data[0].embedding

In [0]:
# <modify> Change to Databricks Mosaic AI Model Serving processing
# client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

client = get_clinet()
embedding_vector = get_embedding(question_text, client=client)

Set the limits for our searches

In [0]:
top_chunks = 3
top_communities = 3
top_outside_relationships = 10
top_inside_relationships = 10
top_entities = 10

## Find nearest Entities
Now we need to use our Elasticsearch index to do a k-nearest neighbour search for our **embedding_vector** to find the 10 nearest `Entity` instances.

In [0]:
# <delete> Change to Mosaic AI Vector Search
# es = Elasticsearch("http://localhost:9200", 
#                    basic_auth=(es_username, es_password), 
#                    verify_certs=False)

In [0]:
# <modify> Change to Mosaic AI Vector Search
# query = {
#     "field" : "description_embedding" ,
#     "query_vector" : embedding_vector,
#     "k" : top_entities,
#     "num_candidates" : 100 ,
# }
# index_name = "entity_graph_index"
# res = es.search(index=index_name, knn=query, source=["id"])
# search_results = res["hits"]["hits"]
# # convert our results into a list of Entities
# # This list will be ordered by match score descending (i.e the more likely matches will be at the beginning)
# entity_list = [x['_id'] for x in search_results]
# entity_list

from databricks.vector_search.client import VectorSearchClient

index_fqn = f"{CATALOG}.{SCHEMA}.{INDEX_NAME}"

vsc = VectorSearchClient()
index = vsc.get_index(index_name=index_fqn)
res = index.similarity_search(
    query_vector=embedding_vector,
    columns=["id"],
    num_results=int(top_entities),
)
rows = res.get("result", {}).get("data_array", [])
entity_list = [row[0] for row in rows]
entity_list

In [0]:
def get_entities(nodes: List[str]) -> str:
    """Get a SPARQL query that will fetch details of the Entites that are in the list
    
    :param nodes: list of Entity ids
    :returns: a SPARQL query string
    """

    query = """
    PREFIX gr: <http://ormynet.com/ns/msft-graphrag#>
    
    SELECT ?id ?description
    WHERE
    {
        ?entity_uri a gr:Entity;
        gr:id ?id;
        gr:description ?entity_desc .
        BIND(REPLACE(?entity_desc, "\\r\\n", " ", "i") AS ?description)
    """
    first = True
    for node in nodes:
        if first:
            query += " FILTER( "
        else:
            query += " || "
        query += f' ?id = "{node}" '
        first = False
    query += """ )
    }
    """
    return query

In [0]:
# Get details about the Entities that were found
entities_df =sparql_query(get_entities(entity_list))
entities_df

In [0]:
def convert_df_to_text(df: pd.DataFrame, key_name: str, colname: str):
    """Convert a DataFrame to text suitable for LLM context
    
    :param df: input DataFrame
    :param key_name: name of the key to use
    :param colname: name of the column in the DataFrame to use
    :returns: string suitable for LLM context
    """
    output_text = "{\"" + key_name + ":\" [\n"
    first = True
    for i in range(len(df)):
        if first:
            output_text += "\""
            first = False
        else:
            output_text += ",\n\"" 
        output_text += df[colname].iloc[i] + "\""
    output_text += "]}"
    return output_text

In [0]:
entity_text = convert_df_to_text(entities_df, 'Entities', 'description')

## Get The Top 3 Chunks
Get the `Chunk` records that are connected to these `Entity` records, sort them by those that have the most Entities and then take the top 3

In [0]:
def get_text_mapping(nodes: List[str], limit_chunks: int = 3) -> str:
    """Get a SPARQL query that fetches the top Chunks that are connected to Entity records
    
    :param nodes: list of Entity ids
    :param limit_chunks: how many chunks to return
    :returns: a SPARQL query string
    """
    query = """
    PREFIX gr: <http://ormynet.com/ns/msft-graphrag#>
    
    SELECT 
    ?chunkText 
    (COUNT(?entity_uri) AS ?freq)
    WHERE {
        ?chunk_uri gr:has_entity ?entity_uri;
        gr:text ?chunk_text .
    """
    first = True
    for node in nodes:
        if not first:
            query += " UNION "
        query += f"""
        {{
            ?entity_uri a gr:Entity;
            gr:id "{node}" .
        }} 
        """
        first = False
    query += """
        BIND(REPLACE(?chunk_text, "\\r\\n", " ") as ?chunkText)
    }
    GROUP BY ?chunk_uri ?chunkText
    ORDER BY DESC(?freq)
    """
    query += f" LIMIT {limit_chunks} "
    return query

In [0]:
# Let's find the Chunks that are most likely to contain the information we're looking for
text_mapping_df = sparql_query(get_text_mapping(entity_list, limit_chunks=top_chunks))
text_mapping_df

In [0]:
chunk_text = convert_df_to_text(text_mapping_df, 'Chunks', 'chunkText')

## Get the Top 3 Relationships
Get the top 3 `Community` records that are related to these `Entity` records.

In [0]:
# <modify> Changed to handle missing weight values so that data can be retrieved (code generated by AI, so there may be behavioral differences)
# def get_report_mapping(nodes: List[str], limit_communities: int = 3) -> str:
#     """Get the Communities that are most likely to contain the Entities
    
#     :param nodes: list of Entity ids
#     :param limit_communities: how many communities
#     :returns: a SPARQL query string
#     """
#     query = """
#     PREFIX gr: <http://ormynet.com/ns/msft-graphrag#>

#     SELECT ?community_uri ?rank ?weight ?summary
#     WHERE
#     {
#         ?community_uri a gr:Community;
#           gr:rank ?rank;
#           gr:weight ?weight;
#           gr:summary ?community_summary .
#         BIND(REPLACE(?community_summary, "\\r\\n", " ", "i") AS ?summary)
#         ?entity_uri gr:in_community ?community_uri;
#     """
#     first = True
#     for node in nodes:
#         if not first:
#             query += " UNION "
#         query += f"""
#         {{
#             ?entity_uri a gr:Entity;
#             gr:id "{node}" .
#         }} 
#         """
#         first = False
#     query += """
#     }
#     GROUP BY ?rank ?weight ?community_uri ?summary
#     ORDER BY DESC(?rank) DESC(?weight)
#     """
#     query += f" LIMIT {limit_communities} "
#     return query

def get_report_mapping(nodes, limit_communities = 3) -> str:
    """
    Generate a SPARQL query that returns communities containing the specified Entity IDs,
    prioritizing those that contain more of the given entities.
    In case of ties, results are ordered by rank and then by weight in descending order.
    """
    if not nodes:
        raise ValueError("At least one node is required.")

    # Remove duplicates while preserving order
    seen, ids = set(), []
    for n in map(str, nodes):
        if n not in seen:
            seen.add(n)
            ids.append(n)

    total = len(ids)

    def esc(s: str) -> str:
        # Simple escape for SPARQL literals
        return s.replace("\\", "\\\\").replace('"', '\\"')

    values_block = "\n    ".join(f'"{esc(i)}"' for i in ids)

    query = f"""
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX gr:  <http://ormynet.com/ns/msft-graphrag#>

    SELECT ?community_uri
        (xsd:decimal(COALESCE(MAX(?rank0), MAX(?level0))) AS ?rank)
        (MAX(?weightN) AS ?weight)
        (REPLACE(STR(SAMPLE(?community_summary)), "[\\\\r\\\\n]+", " ") AS ?summary)
        (COUNT(DISTINCT ?id) AS ?match_count)
        ((xsd:decimal(COUNT(DISTINCT ?id)) / xsd:decimal({total})) AS ?coverage)
    WHERE {{
    VALUES ?id {{
        {values_block}
    }}

    ?entity_uri a gr:Entity ;
                gr:id ?id ;
                gr:in_community ?community_uri .

    ?community_uri a gr:Community .

    OPTIONAL {{ ?community_uri gr:rank  ?rank0 }}
    OPTIONAL {{ ?community_uri gr:level ?level0 }}

    OPTIONAL {{
        ?community_uri gr:weight ?weight0 .
        BIND(xsd:decimal(?weight0) AS ?weightN)
    }}

    OPTIONAL {{ ?community_uri gr:summary ?community_summary }}
    }}
    GROUP BY ?community_uri
    ORDER BY DESC(?match_count) DESC(?rank) DESC(?weight)
    LIMIT {limit_communities}
    """.strip()

    return query


In [0]:
# Get the top communities that these Entities are part of
report_mapping_df = sparql_query(get_report_mapping(entity_list, limit_communities=top_communities))
report_mapping_df

In [0]:
reports_text = convert_df_to_text(report_mapping_df, 'Reports', 'summary')

## Get The Outside & Inside Relationships
Get the outside and inside relationsihps for the `Entity` records.

In [0]:
def get_outside_relationships(nodes: List[str], limit_outside_relationships: int = 10) -> str:
    """Get the outside relationships
    
    :param nodes: list of Entity ids
    :param limit_outside_relationships: how many relationships to return
    :returns: a SPARQL query string
    """
    query = """
    PREFIX gr: <http://ormynet.com/ns/msft-graphrag#>
    
    SELECT 
    ?description
    ?entity_from_id ?entity_to_id
    ?rank ?weight
    WHERE {
        ?related_to_uri a gr:related_to;
            gr:id ?id;
            gr:rank ?rank;
            gr:description ?desc;
            gr:weight ?weight .
        BIND(REPLACE(?desc, "\\r\\n", "") as ?description)
        ?entity_from_uri ?related_to_uri ?entity_to_uri .
        ?entity_from_uri gr:id ?entity_from_id .
        ?entity_to_uri gr:id ?entity_to_id .
    """
    first = True
    for node in nodes:
        if first:
            query += " FILTER( "
        else:
            query += " && "
        query += f"""
    ?entity_to_id != "{node}" """
        first = False
    query += """
               )
    }
    ORDER BY DESC(?rank) DESC(?weight)
    """
    query += f" LIMIT {limit_outside_relationships} "
    return query

In [0]:
# Get the top outside relationships these Entities are not part of
outside_relationships_df = sparql_query(get_outside_relationships(entity_list, limit_outside_relationships=top_outside_relationships))
outside_relationships_df

In [0]:
def get_inside_relationships(nodes: List[str], limit_inside_relationships: int = 10) -> str:
    """Get a SPARQL query to fetch the inside relationships
    
    :param nodes: list of Entity ids
    :param limit_inside_relationships: how many relationships to return
    :returns: a SPARQL query string
    """
    query = """
    PREFIX gr: <http://ormynet.com/ns/msft-graphrag#>
    SELECT 
    ?description
    ?entity_from_id ?entity_to_id
    ?rank ?weight
    WHERE {
        ?related_to_uri a gr:related_to;
            gr:id ?id;
            gr:rank ?rank;
            gr:description ?desc;
            gr:weight ?weight .
        BIND(REPLACE(?desc, "\\r\\n", "") as ?description)
        ?entity_from_uri ?related_to_uri ?entity_to_uri .
        ?entity_from_uri gr:id ?entity_from_id .
        ?entity_to_uri gr:id ?entity_to_id .
    """
    first = True
    for node in nodes:
        if first:
            query += " FILTER( "
        else:
            query += " || "
        query += f"""
        ?entity_to_id = "{node}" """
        first = False
    query += """
               )
    }
    ORDER BY DESC(?rank) DESC(?weight)
    """
    query += f" LIMIT {limit_inside_relationships} "
    return query

In [0]:
# Get the top inside relationships these Entities are part of
inside_relationships_df = sparql_query(get_inside_relationships(entity_list, limit_inside_relationships=top_inside_relationships))
inside_relationships_df

In [0]:
relationships_text = "{\"Relationships:\" [ "
first = True
for i in range(len(inside_relationships_df)):
    if first:
        relationships_text += "\""
        first = False
    else:
        relationships_text += ",\n\"" 
    relationships_text += inside_relationships_df['description'].iloc[i] + "\""
for i in range(len(outside_relationships_df)):
    relationships_text += outside_relationships_df['description'].iloc[i] + "\""
relationships_text += "]}"

## Create LangChain Response
Having got all our important data for our identified entity list, we now need to combine them to produce a response that would be suitable as a LangChain response.

In [0]:
# <modify>  Change to Databricks Mosaic AI Model Serving processing
# llm = ChatOpenAI(
#     model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
#     api_key="lm-server",
#     base_url="http://localhost:1234/v1"
# )

from databricks_langchain.chat_models import ChatDatabricks

llm = ChatDatabricks(
    endpoint="databricks-meta-llama-3-1-8b-instruct",
    temperature=None,
)

## Prompt with no context
Prompt with no context fromt the Knowledge Graph

In [0]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that answers questions about a book.",
        ),
        ("human", "{input}"),
    ]
)
chain = prompt | llm | StrOutputParser()
chain.invoke(
    {
        "input": question_text,
    }
)

## Prompt with context from Knowledge Graph
Create a context using the Knowledge Graph and feed that to the LLM.

In [0]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that answers questions about a book.",
        ),
        ("human", "{context} {input}"),
    ]
)
chain = prompt | llm | StrOutputParser()
chain.invoke(
    {
        "context": entity_text + "," + chunk_text + "," + relationships_text +"," + reports_text,
        "input": question_text,
    }
)

## Global Query
We'll setup a global query with 2 different prompt templates.

In [0]:
MAP_SYSTEM_PROMPT = """
---Role---

You are a helpful assistant responding to questions about data in the tables provided.


---Goal---

Generate a response consisting of a list of key points that responds to the user's question, summarizing all relevant information in the input data tables.

You should use the data provided in the data tables below as the primary context for generating the response.
If you don't know the answer or if the input data tables do not contain sufficient information to provide an answer, just say so. Do not make anything up.

Each key point in the response should have the following element:
- Description: A comprehensive description of the point.
- Importance Score: An integer score between 0-100 that indicates how important the point is in answering the user's question. An 'I don't know' type of response should have a score of 0.

The response should be JSON formatted as follows:
{{
    "points": [
        {{"description": "Description of point 1 [Data: Reports (report ids)]", "score": score_value}},
        {{"description": "Description of point 2 [Data: Reports (report ids)]", "score": score_value}}
    ]
}}

The response shall preserve the original meaning and use of modal verbs such as "shall", "may" or "will".

Points supported by data should list the relevant reports as references as follows:
"This is an example sentence supported by data references [Data: Reports (report ids)]"

**Do not list more than 5 record ids in a single reference**. Instead, list the top 5 most relevant record ids and add "+more" to indicate that there are more.

For example:
"Person X is the owner of Company Y and subject to many allegations of wrongdoing [Data: Reports (2, 7, 64, 46, 34, +more)]. He is also CEO of company X [Data: Reports (1, 3)]"

where 1, 2, 3, 7, 34, 46, and 64 represent the id (not the index) of the relevant data report in the provided tables.

Do not include information where the supporting evidence for it is not provided.


---Data tables---

{context_data}

---Goal---

Generate a response consisting of a list of key points that responds to the user's question, summarizing all relevant information in the input data tables.

You should use the data provided in the data tables below as the primary context for generating the response.
If you don't know the answer or if the input data tables do not contain sufficient information to provide an answer, just say so. Do not make anything up.

Each key point in the response should have the following element:
- Description: A comprehensive description of the point.
- Importance Score: An integer score between 0-100 that indicates how important the point is in answering the user's question. An 'I don't know' type of response should have a score of 0.

The response shall preserve the original meaning and use of modal verbs such as "shall", "may" or "will".

Points supported by data should list the relevant reports as references as follows:
"This is an example sentence supported by data references [Data: Reports (report ids)]"

**Do not list more than 5 record ids in a single reference**. Instead, list the top 5 most relevant record ids and add "+more" to indicate that there are more.

For example:
"Person X is the owner of Company Y and subject to many allegations of wrongdoing [Data: Reports (2, 7, 64, 46, 34, +more)]. He is also CEO of company X [Data: Reports (1, 3)]"

where 1, 2, 3, 7, 34, 46, and 64 represent the id (not the index) of the relevant data report in the provided tables.

Do not include information where the supporting evidence for it is not provided.

The response should be JSON formatted as follows:
{{
    "points": [
        {{"description": "Description of point 1 [Data: Reports (report ids)]", "score": score_value}},
        {{"description": "Description of point 2 [Data: Reports (report ids)]", "score": score_value}}
    ]
}}
"""
map_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            MAP_SYSTEM_PROMPT,
        ),
        (
            "human",
            "{question}",
        ),
    ]
)
map_chain = map_prompt | llm | StrOutputParser()

In [0]:
REDUCE_SYSTEM_PROMPT = """
---Role---

You are a helpful assistant responding to questions about a dataset by synthesizing perspectives from multiple analysts.


---Goal---

Generate a response of the target length and format that responds to the user's question, summarize all the reports from multiple analysts who focused on different parts of the dataset.

Note that the analysts' reports provided below are ranked in the **descending order of importance**.

If you don't know the answer or if the provided reports do not contain sufficient information to provide an answer, just say so. Do not make anything up.

The final response should remove all irrelevant information from the analysts' reports and merge the cleaned information into a comprehensive answer that provides explanations of all the key points and implications appropriate for the response length and format.

Add sections and commentary to the response as appropriate for the length and format. Style the response in markdown.

The response shall preserve the original meaning and use of modal verbs such as "shall", "may" or "will".

The response should also preserve all the data references previously included in the analysts' reports, but do not mention the roles of multiple analysts in the analysis process.

**Do not list more than 5 record ids in a single reference**. Instead, list the top 5 most relevant record ids and add "+more" to indicate that there are more.

For example:

"Person X is the owner of Company Y and subject to many allegations of wrongdoing [Data: Reports (2, 7, 34, 46, 64, +more)]. He is also CEO of company X [Data: Reports (1, 3)]"

where 1, 2, 3, 7, 34, 46, and 64 represent the id (not the index) of the relevant data record.

Do not include information where the supporting evidence for it is not provided.


---Target response length and format---

{response_type}


---Analyst Reports---

{report_data}


---Goal---

Generate a response of the target length and format that responds to the user's question, summarize all the reports from multiple analysts who focused on different parts of the dataset.

Note that the analysts' reports provided below are ranked in the **descending order of importance**.

If you don't know the answer or if the provided reports do not contain sufficient information to provide an answer, just say so. Do not make anything up.

The final response should remove all irrelevant information from the analysts' reports and merge the cleaned information into a comprehensive answer that provides explanations of all the key points and implications appropriate for the response length and format.

The response shall preserve the original meaning and use of modal verbs such as "shall", "may" or "will".

The response should also preserve all the data references previously included in the analysts' reports, but do not mention the roles of multiple analysts in the analysis process.

**Do not list more than 5 record ids in a single reference**. Instead, list the top 5 most relevant record ids and add "+more" to indicate that there are more.

For example:

"Person X is the owner of Company Y and subject to many allegations of wrongdoing [Data: Reports (2, 7, 34, 46, 64, +more)]. He is also CEO of company X [Data: Reports (1, 3)]"

where 1, 2, 3, 7, 34, 46, and 64 represent the id (not the index) of the relevant data record.

Do not include information where the supporting evidence for it is not provided.


---Target response length and format---

{response_type}

Add sections and commentary to the response as appropriate for the length and format. Style the response in markdown.
"""

reduce_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            REDUCE_SYSTEM_PROMPT,
        ),
        (
            "human",
            "{question}",
        ),
    ]
)
reduce_chain = reduce_prompt | llm | StrOutputParser()

In [0]:
def global_retriever(query: str, level: int, response_type: str = "multiple paragraphs") -> str:
    """Global retriever

    :param query: the question as string
    :param level: the Community level
    :param response_type: type of response
    :returns: final response as a string
    """
    community_query = f"""
    PREFIX gr: <http://ormynet.com/ns/msft-graphrag#>
    
    SELECT ?full_content
    WHERE {{
        ?community_uri a gr:Community;
        gr:level ?level;
        gr:full_content ?full_content .
        FILTER(?level = {level})
    }}
    """
    community_data = sparql_query(community_query)
    intermediate_results = []
    for i in tqdm(range(len(community_data)), desc="Processing communities"):
        intermediate_response = map_chain.invoke(
            {"question": query, "context_data": community_data["full_content"].iloc[i]}
        )
        intermediate_results.append(intermediate_response)
    final_response = reduce_chain.invoke(
        {
            "report_data": intermediate_results,
            "question": query,
            "response_type": response_type,
        }
    )
    return final_response

In [0]:
print(global_retriever("What is the story about?", 2))